In [21]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import os
from geopy.distance import geodesic
import requests
import time

## Extracción de datos históricos de los incendios por fecha de AEMET

Cargamos la última versión del dataset

In [19]:
bdif = pd.read_excel("bdif_geograficas.xlsx")
bdif.head()

,parte,año,cod_com,cod_prov,probignicion,diastormenta,diasultimalluvia,tempmaxima,humrelativa,velocidadviento,...,mes,tipodia,id_rel,provincia,poblacion,superficie,altitud,areaquemada,lon,lat
0,1974020249,1974,11,2,0.0,NaN,NaN,10.0,75.0,30.0,...,abril,laborable,1020864,Albacete,2478.0,51221.68,878.0,8.0,-2.318960,38.366800
1,1974020374,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,mayo,festivo,1020371,Albacete,30516.0,77929.12,570.0,0.3,-1.703449,38.512190
2,1974020459,1974,11,46,0.0,NaN,NaN,NaN,NaN,NaN,...,junio,laborable,1460446,València/Valencia,5223.0,44668.00,596.0,0.6,-1.056085,39.059798
3,1974022274,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,septiembre,laborable,1020674,Albacete,1326.0,8100.28,1126.0,2.0,-2.418393,38.499810
4,1974022457,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,octubre,laborable,1020117,Albacete,601.0,14658.96,696.0,0.5,-2.070538,38.552177


In [58]:
bdif.shape

(588555, 26)

Extraemos las fechas de los incendios

In [20]:
# Guardamos las fechas de detección de los incendios
fechas = pd.to_datetime(bdif['deteccion'])
# Extraemos solo la fecha
fechas = fechas.dt.date
# Eliminamos los duplicados 
fechas = fechas.drop_duplicates()
fechas.head()

0    1974-04-25
1    1974-05-19
2    1974-06-11
3    1974-09-11
4    1974-10-01
Name: deteccion, dtype: object

Definimos la clave API de AEMET y configuramos la ruta de alamacenamiento

In [28]:
# API key de AEMET
api_key = ''
# Ruta a la carpeta de almacenamiento
directorio = 'meteo/'

Extraemos los datos

In [25]:
# Función para extraer los datos
def extraer_meteo(fecha):
    """
    Función para extraer los datos meteorológicos diarios de AEMET para una fecha específica.
    
    Esta función realiza una solicitud a la API de AEMET para obtener datos climatológicos
    de una fecha de todas las estaciones meteorológicas.
    Los datos se descargan en formato JSON y se guardan con la fecha como nombre.
    
    Parámetros:
    fecha (str): Fecha del incendio en formato 'YYYY-MM-DD'.

    La función hace lo siguiente:
    1. Construye la fecha de inicio y fin con formato específico requerido por la API de AEMET.
    2. Realiza una solicitud a la API de AEMET con las fechas de inicio y fin para obtener los enlaces de los datos.
       Como las fechas de inicio y fin son iguales, se ajusta la hora para realizar la consulta correctamente.
    3. Descarga los datos en formato JSON.
    4. Guarda los datos descargados en un archivo con el nombre correspondiente a la fecha.
    
    Retorna:
    Se van guardando los archivos y se obtienen mensajes por consola.
    """
    # Montamos la fecha de petición en el formato que se solicita
    fechaini = str(fecha) + 'T00:00:00UTC'
    fechafin = str(fecha) + 'T23:59:59UTC'
    
    # Realizamos la solicitud
    url = f'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/{fechaini}/fechafin/{fechafin}/todasestaciones'
    headers = {'cache-control': "no-cache"}
    response = requests.request("GET", url, headers=headers, params={"api_key":api_key})

    # Comprobamos que la respuesta sea ok
    if response.status_code == 200:
        # Obtener el enlace de los datos
        data_url = response.json().get('datos', '')

        # Descargar el json
        if data_url:
            data_response = requests.get(data_url)
            if data_response.status_code == 200:
                # Guardar el json con la fecha como nombre
                file_path = f"{directorio}/{str(fecha)}.json"
                with open(file_path, 'w') as f:
                    f.write(data_response.text)
                print(f'Datos descargados para la fecha {str(fecha)}')
            else:
                print(f'Error al descargar los datos para la fecha {str(fecha)}')
        else:
            print(f'No se encontraron datos para la fecha {str(fecha)}')
    else:
        print(f'Error en la solicitud para la fecha {str(fecha)}: {response.status_code}')

In [ ]:
# Limitamos la conexión porque solo podemos hacer 50 peticiones por minuto
for fecha in fechas:
    extraer_meteo(fecha)
    time.sleep(2)

## Listado de estaciones meteorológicas

In [13]:
# Cargamos el archivo
estaciones = pd.read_json('estaciones_meteorologicas.json')

# Mostramos los primeros registros
estaciones.head()

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
0,394924N,ILLES BALEARS,490,B013X,"ESCORCA, LLUC",08304,025309E
1,394744N,ILLES BALEARS,5,B051A,"SÓLLER, PUERTO",08316,024129E
2,394121N,ILLES BALEARS,60,B087X,BANYALBUFAR,,023046E
3,393446N,ILLES BALEARS,52,B103B,ANDRATX - SANT ELM,,022208E
4,393305N,ILLES BALEARS,50,B158X,"CALVIÀ, ES CAPDELLÀ",,022759E


In [14]:
# Vamos a extraer el indicativo y las coordenadas de cada estación meteorológica
estaciones = estaciones[['indicativo', 'latitud', 'longitud']]
estaciones.head()

,indicativo,latitud,longitud
0,B013X,394924N,025309E
1,B051A,394744N,024129E
2,B087X,394121N,023046E
3,B103B,393446N,022208E
4,B158X,393305N,022759E


In [15]:
# Necesitamos las coordenadas en grados
# Creamos una función de transformación
def conversion_a_grados(coordenada):
    """
    Dada una coordenada en grados, minutos y segundos, la transformamos a grados decimales.

    La entrada es una cadena de texto que tiene la siguiente estructura:
    - Los dos primeros numeros que representan los grados.
    - Los dos números centrales que representan los minutos.
    - Los dos últimos números representan los segundos.
    - La letra final que representa la orientación: 'N', 'S', 'E' o 'W'.
    
    Ejemplo de entrada: '394924N'

    La fórmula que realiza la conversión es la siguiente:
    grados + (minutos / 60) + (segundos / 3600).

    Si tenemos orientación 'S' o 'W' el valor será negativo.

    Parámetros:
    coordenada (str): coordenada en grados, minutos y segundos más la orientación.

    Retorna:
    float: La coordenada en grados decimales.

    Ejemplo:
    conversion_a_grados('394924N') -> 39.8233333
    conversion_a_grados('074550W') -> -74.9166667
    """
    g = int(coordenada[:2])
    m = int(coordenada[2:4])
    s = int(coordenada[4:6])  
    o = coordenada[6]
    gd = g + (m/60) + (s/3600)
    if o in ['S', 'W']:
        gd = -gd
    return gd

In [16]:
# Le pasamos la función a las columnas de latitud y longitud
estaciones['latitud'] = estaciones['latitud'].apply(conversion_a_grados)
estaciones['longitud'] = estaciones['longitud'].apply(conversion_a_grados)

# Mostramos el dataframe para comprobar los cambios
estaciones.head()

,indicativo,latitud,longitud
0,B013X,39.823333,2.885833
1,B051A,39.795556,2.691389
2,B087X,39.689167,2.512778
3,B103B,39.579444,2.368889
4,B158X,39.551389,2.466389


## Búsqueda de la estación más cercana

In [43]:
# Creamos la función que dada unas coordenadas encuentre las estaciones
# meteorológica más cercanas por orden de proximidad
def estacion_cercana(latitud, longitud):
    """
    Encuentra las estaciones meteorológicas más cercanas dadas unas coordenadas geográficas.

    Calcula la distancia entre las coordenadas dadas y las de cada estación y devuelve
    un dataframe con las estaciones ordenadas por distancia ascendente.

    Parámetros:
        latitud (float): Latitud de la ubicación del incendio.
        longitud (float): Longitud de la ubicación del incendio.

    Retorna:
        pandas.DataFrame: DataFrame con las estaciones más cercanas. Incluye:
            - 'indicativo': un código que identifica la estación.
            - 'distancia': la distancia en km de las coordenadas a la estación.
    """
    estaciones_cercanas = []
    for i, f in estaciones.iterrows():
        # Calculo de la distancia entre las coordenadas iniciales y la estación correspondiente
        distancia = geodesic( (latitud,longitud),(f['latitud'],f['longitud'])).kilometers

        # Vamos guardando las estaciones y las distancias
        estaciones_cercanas.append({
            'indicativo': f['indicativo'],
            'distancia': distancia
        })

    # Lo pasamos a dataframe
    df_estaciones_cercanas = pd.DataFrame(estaciones_cercanas)

    # Lo ordenamos por distancia ascendente
    df_estaciones_cercanas = df_estaciones_cercanas.sort_values(by='distancia', ascending=True).reset_index(drop=True)

    return df_estaciones_cercanas

## Extracción de los datos meteorológicos

In [45]:
# Creamos una función que dada una fecha y unas coordenadas,
# extraiga los datos meteorológicos de la carpeta donde se han guardado
# el historico de datos climatologicos de las fechas de los incendios

def extraer_datos (fecha, indicativo):
    """
    Extrae los datos meteorológicos de una estación específica de un archivo JSON para una fecha dada.

    Args:
        fecha (str): Fecha del archivo JSON a leer (formato YYYY-MM-DD).
        indicativo (str): Indicativo de la estación meteorológica.

    Returns:
        dict: Datos meteorológicos de la estación si se encuentra, o None si no se encuentra.
    """
    # Ruta al archivo
    archivo = os.path.join('meteo', f'{fecha}.json')
    # Leemos el json y cargamos los datos
    datos = pd.read_json(archivo,  encoding='latin-1')
    
    # Buscamos si la estación está en el archivo
    for _,estacion in datos.iterrows():
        if estacion['indicativo'] == indicativo:
            
            # Convertir la fila de la estación en un diccionario
            estacion = estacion.to_dict()  # Convertir la fila en un diccionario
            
            # Convertir las claves del diccionario a minúsculas
            estacion = {k.lower(): v for k, v in estacion.items()}
            
            # Extraer los datos meteorológicos de la estación
            meteo = {"tmed": estacion.get("tmed", ""),
                    "prec": estacion.get("prec", ""),
                    "tmin": estacion.get("tmin", ""),
                    "tmax": estacion.get("tmax", ""),
                    "dir": estacion.get("dir", ""),
                    "velmedia": estacion.get("velmedia", ""),
                    "racha": estacion.get("racha", ""),
                    "sol": estacion.get("sol", ""),
                    "presmax": estacion.get("presmax", ""),
                    "presmin": estacion.get("presmin", ""),
                    "hrmedia": estacion.get("hrmedia", ""),
                    "hrmax": estacion.get("hrmax", ""),
                    "hrmin": estacion.get("hrmin", "")}
            return meteo
    return None

In [79]:
# Ahora cargamos el archivo con el numero de parte, fecha y coordenadas
partes = bdif[['parte', 'deteccion', 'lon', 'lat']].sort_values(by='deteccion', ascending=True)
partes.head()

,parte,deteccion,lon,lat
2466,1974330044,1974-01-03 19:00:00,-5.663215,43.394746
1283,1974200015,1974-01-04 13:00:00,-1.948745,43.281484
245,1974080001,1974-01-04 16:00:00,1.767066,41.680295
1282,1974200014,1974-01-04 18:00:00,-1.948745,43.281484
2467,1974330045,1974-01-04 22:00:00,-6.414982,43.335256


## Selección de fechas

Elegimos el intervalo de fechas a descargar, por ejemplo, de año en año

In [80]:
# Filtro de fechas
inicio = pd.to_datetime('2021-01-01')
fin = pd.to_datetime('2021-12-31')

# Convertir la columna 'fecha' a formato datetime
partes['deteccion'] = pd.to_datetime(partes['deteccion'])

# Filtrar por rango
filtro = (partes['deteccion'] >= inicio) & (partes['deteccion'] <= fin)
partes = partes[filtro]
partes.head()

,parte,deteccion,lon,lat
588427,2021180001,2021-01-13 15:10:00,-3.438754,37.159556
588042,2021060001,2021-01-19 10:05:00,-5.741597,38.645215
588428,2021180002,2021-01-23 13:45:00,-3.390769,36.743235
588429,2021180003,2021-01-24 14:27:00,-3.284614,37.276822
588512,2021420001,2021-01-27 11:35:00,-2.519507,41.189623


In [81]:
clima = []
start_time = time.time()  # Guardamos el tiempo inicial

# Vamos recorriendo los incendios
for _, parte in partes.iterrows():
    # Creamos el registro del incendio
    registro_clima = {'parte': parte['parte'], 'indicativo': indicativo}
    
    # Tomamos la fecha, longitud y latitud
    fecha = pd.to_datetime(parte['deteccion']).strftime('%Y-%m-%d')
    lon = parte['lon']
    lat = parte['lat']
    
    # Obtenemos el listado de estaciones más cercanas al incendio
    lista_estaciones_cercanas = estacion_cercana(lat, lon)

    # Extraemos la información
    for _, estacion in lista_estaciones_cercanas.iterrows():
        indicativo = estacion['indicativo']
        meteo = extraer_datos(fecha, indicativo)
        # En el momento que tiene la información sale
        if meteo:
            break

    # Agregamos los datos al registro creado
    registro_clima.update(meteo)

    # Añadir el registro a la lista
    clima.append(registro_clima)


# Convertir la lista en dataframe
bdif_climatologicas = pd.DataFrame(clima)
bdif_climatologicas

end_time = time.time()  # Guardamos el tiempo final

execution_time = end_time - start_time  # Calculamos el tiempo de ejecución
print(f"Tiempo de ejecución: {execution_time} segundos")

Tiempo de ejecución: 155.01646208763123 segundos


In [82]:
# Extraemos los datos a excel
bdif_climatologicas.to_excel("variables climatologicas/bdif_climatologicas_2021.xlsx", index=False)

## Fusión de archivos

In [83]:
# Listado de años
años = range(1974, 2021)

dataframes = []

# Vamos cargando los archivos
for año in años:
    # Montamos la ruta al archivo
    archivo = f'variables climatologicas/bdif_climatologicas_{año}.xlsx'
    
    df_año = pd.read_excel(archivo)
        
    # Añadir el DataFrame cargado a la lista
    dataframes.append(df_año)

# Concatenamos todos
bdif_climatologicas = pd.concat(dataframes, ignore_index=True)

# Mostrar el DataFrame concatenado
bdif_climatologicas.head()

,parte,indicativo,tmed,prec,tmin,tmax,dir,velmedia,racha,sol,presmax,presmin,hrmedia,hrmax,hrmin
0,1974330044,1208A,"10,6","0,4","7,0","14,2",27.0,"2,5","5,6","5,2","1008,4","995,4",60.0,NaN,NaN
1,1974200015,1024E,"9,2","0,0","6,1","12,2",18.0,"12,2","25,3","3,9","985,1","980,6",52.0,NaN,NaN
2,1974080001,0158O,"5,5","0,0","2,0","9,0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1974200014,1024E,"9,2","0,0","6,1","12,2",18.0,"12,2","25,3","3,9","985,1","980,6",52.0,NaN,NaN
4,1974330036,1212E,"10,8","1,7","4,1","17,6",NaN,"3,9",NaN,"1,3","993,9","985,9",47.0,NaN,NaN


In [84]:
bdif_climatologicas.to_excel('bdif_climatologicas.xlsx', index=False)
print(f"El archivo ha sido guardado como {archivo_salida}")

El archivo ha sido guardado como bdif_climatologicas.xlsx


In [85]:
# Cargar el dataset y mostrar las primeras filas
bdif_geograficas = pd.read_excel("bdif_geograficas.xlsx")
bdif_geograficas.head()

,parte,año,cod_com,cod_prov,probignicion,diastormenta,diasultimalluvia,tempmaxima,humrelativa,velocidadviento,...,mes,tipodia,id_rel,provincia,poblacion,superficie,altitud,areaquemada,lon,lat
0,1974020249,1974,11,2,0.0,NaN,NaN,10.0,75.0,30.0,...,abril,laborable,1020864,Albacete,2478.0,51221.68,878.0,8.0,-2.318960,38.366800
1,1974020374,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,mayo,festivo,1020371,Albacete,30516.0,77929.12,570.0,0.3,-1.703449,38.512190
2,1974020459,1974,11,46,0.0,NaN,NaN,NaN,NaN,NaN,...,junio,laborable,1460446,València/Valencia,5223.0,44668.00,596.0,0.6,-1.056085,39.059798
3,1974022274,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,septiembre,laborable,1020674,Albacete,1326.0,8100.28,1126.0,2.0,-2.418393,38.499810
4,1974022457,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,octubre,laborable,1020117,Albacete,601.0,14658.96,696.0,0.5,-2.070538,38.552177


In [74]:
# Cargar el dataset y mostrar las primeras filas
bdif_climatologicas = pd.read_excel("bdif_climatologicas.xlsx")
bdif_climatologicas.head()

,parte,fecha,lon,lat,indicativo,tmed,prec,tmin,tmax,velmedia,sol,presMax,horaPresMax,presMin,horaPresMin,hrMedia
0,1974330044,1974-01-03,-5.663215,43.394746,1208A,"10,6","0,4","7,0","14,2","2,5","5,2","1008,4",24,"995,4",06,60.0
1,1974200015,1974-01-04,-1.948745,43.281484,1024E,"9,2","0,0","6,1","12,2","12,2","3,9","985,1",11,"980,6",24,52.0
2,1974080001,1974-01-04,1.767066,41.680295,0158O,"5,5","0,0","2,0","9,0",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1974200014,1974-01-04,-1.948745,43.281484,1024E,"9,2","0,0","6,1","12,2","12,2","3,9","985,1",11,"980,6",24,52.0
4,1974330036,1974-01-04,-6.535122,43.544528,1212E,"10,8","1,7","4,1","17,6","3,9","1,3","993,9",00,"985,9",24,47.0


In [86]:
# Unimos los dataframes
bdif = pd.merge( bdif_geograficas, bdif_climatologicas, on='parte', how='left')

# Mostramos las primeras filas como comprobación
bdif.head()

,parte,año,cod_com,cod_prov,probignicion,diastormenta,diasultimalluvia,tempmaxima,humrelativa,velocidadviento,...,tmax,dir,velmedia,racha,sol,presmax,presmin,hrmedia,hrmax,hrmin
0,1974020249,1974,11,2,0.0,NaN,NaN,10.0,75.0,30.0,...,"17,0",29.0,"6,1","11,9","9,5","933,9","927,9",56.0,NaN,NaN
1,1974020374,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,"26,6",18.0,"6,4","13,1","10,9","938,8","935,9",53.0,NaN,NaN
2,1974020459,1974,11,46,0.0,NaN,NaN,NaN,NaN,NaN,...,"23,0",9.0,"3,6","11,1","3,3","1015,5","1011,1",71.0,NaN,NaN
3,1974022274,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,"28,0",NaN,"5,0",NaN,"8,9","940,9","937,9",63.0,NaN,NaN
4,1974022457,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,"23,2",NaN,"4,7",NaN,"9,3","941,3","937,6",45.0,NaN,NaN


In [87]:
bdif.to_excel('bdif_geo_clim.xlsx', index=False)
print(f"El archivo ha sido guardado como bdif_geo_clim")

El archivo ha sido guardado como bdifbueno.xlsx


In [88]:
bdif.shape

(588555, 40)

In [89]:
bdif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588555 entries, 0 to 588554
Data columns (total 40 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   parte             588555 non-null  int64         
 1   año               588555 non-null  int64         
 2   cod_com           588555 non-null  int64         
 3   cod_prov          588555 non-null  int64         
 4   probignicion      475296 non-null  float64       
 5   diastormenta      24315 non-null   float64       
 6   diasultimalluvia  223547 non-null  float64       
 7   tempmaxima        296771 non-null  float64       
 8   humrelativa       291606 non-null  float64       
 9   velocidadviento   276913 non-null  float64       
 10  direccionviento   218404 non-null  float64       
 11  detectadopor      588055 non-null  object        
 12  deteccion         588555 non-null  datetime64[ns]
 13  combustible       588555 non-null  object        
 14  tipo

### Licencias de uso de datos

Los datos utilizados en este análisis provienen de los siguientes recursos:

- **Estadística General de Incendios Forestales (EGIF)**, gestionada por el Ministerio para la Transición Ecológica y el Reto Demográfico (MITECO). Los datos deben ser utilizados de acuerdo con las [condiciones de uso de la EGIF](https://www.miteco.gob.es/es/biodiversidad/temas/incendios-forestales/estadisticas-datos.aspx).
- **AEMET** y están sujetos a las [condiciones de uso de AEMET](https://www.aemet.es/es/serviciosclimaticos/datos).
- **Nomenclátor Geográfico de Municipios y Entidades de Población (NGMEP)**, proporcionado por el **Centro Nacional de Información Geográfica (CNIG)**. Los datos están sujetos a la licencia de [Datos Abiertos del CNIG](https://astronoomia.ign.es/web/ign/portal). Se debe proporcionar citación adecuada al utilizar los datos.

Recuerda que es importante respetar las licencias de uso al compartir o distribuir los resultados de este notebook.
